In [ ]:
!pip install gensim
# Made with help from GPT
import nltk
import random
from nltk.corpus import movie_reviews
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
import numpy as np
from typing import List, Optional, Any


# Download required NLTK data
nltk.download('movie_reviews')
# nltk.download('punkt')


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:

# Load and shuffle documents
documents : list[tuple[list[str], str]]  = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

# Extract only the review texts for training Word2Vec
sentences = [list(map(str.lower, movie_reviews.words(fileid)))
             for fileid in movie_reviews.fileids()]


In [3]:
# Train Word2Vec model (Skip-Gram)
embedding_dim = 200
w2v_model = Word2Vec(
    sentences,
    vector_size=embedding_dim,
    window=15,
    min_count=2,
    sg=1,  # Skip-Gram
    workers=8,
    epochs=10
)


In [4]:

def get_vector(word: str) -> Optional[np.ndarray]:
    """Fetch vector from Word2Vec model if word exists."""
    return w2v_model.wv[word] if word in w2v_model.wv else None

def get_review_vector_avg(words: List[str]) -> np.ndarray:
    """Compute average vector for a list of words."""
    vectors = [w2v_model.wv[w] for w in map(str.lower, words) if w in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(embedding_dim, dtype=np.float32)

def get_review_vector_max(words: List[str]) -> np.ndarray:
    """Compute max-pooled vector for a list of words."""
    vectors = [w2v_model.wv[w] for w in map(str.lower, words) if w in w2v_model.wv]
    return np.max(vectors, axis=0) if vectors else np.zeros(embedding_dim, dtype=np.float32)



# Prepare data
X_avg = []
X_max = []
y = []

for words, label in documents:
    X_avg.append(get_review_vector_avg(words))
    X_max.append(get_review_vector_max(words))
    y.append(1 if label == 'pos' else 0)


In [5]:

# Split data
X_avg_train, X_avg_test, y_train, y_test = train_test_split(X_avg, y, test_size=0.2, random_state=42)
X_max_train, X_max_test, _, _ = train_test_split(X_max, y, test_size=0.2, random_state=42)

# Train classifiers
clf_avg = LogisticRegression(max_iter=1000)
clf_avg.fit(X_avg_train, y_train)
y_pred_avg = clf_avg.predict(X_avg_test)

clf_max = LogisticRegression(max_iter=1000)
clf_max.fit(X_max_train, y_train)
y_pred_max = clf_max.predict(X_max_test)

# Results
print("Classification Report (Average Vector):")
print(classification_report(y_test, y_pred_avg))

print("Classification Report (Max Pooling Vector):")
print(classification_report(y_test, y_pred_max))

Classification Report (Average Vector):
              precision    recall  f1-score   support

           0       0.80      0.86      0.83       199
           1       0.85      0.78      0.82       201

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

Classification Report (Max Pooling Vector):
              precision    recall  f1-score   support

           0       0.70      0.73      0.72       199
           1       0.72      0.70      0.71       201

    accuracy                           0.71       400
   macro avg       0.71      0.71      0.71       400
weighted avg       0.71      0.71      0.71       400



In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

scale_data = True

# Split data
X_avg_train, X_avg_test, y_train, y_test = train_test_split(X_avg, y, test_size=0.2, random_state=42)
X_max_train, X_max_test, _, _ = train_test_split(X_max, y, test_size=0.2, random_state=42)

if scale_data:
    clf_avg = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
    clf_max = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
else:
    clf_avg = LogisticRegression(max_iter=1000)
    clf_max = LogisticRegression(max_iter=1000)

# Train classifiers
clf_avg.fit(X_avg_train, y_train)
y_pred_avg = clf_avg.predict(X_avg_test)

clf_max.fit(X_max_train, y_train)
y_pred_max = clf_max.predict(X_max_test)

# Results
print("Classification Report (Average Vector):")
print(classification_report(y_test, y_pred_avg))

print("Classification Report (Max Pooling Vector):")
print(classification_report(y_test, y_pred_max))


Classification Report (Average Vector):
              precision    recall  f1-score   support

           0       0.92      0.86      0.89       199
           1       0.87      0.93      0.90       201

    accuracy                           0.89       400
   macro avg       0.89      0.89      0.89       400
weighted avg       0.89      0.89      0.89       400

Classification Report (Max Pooling Vector):
              precision    recall  f1-score   support

           0       0.68      0.71      0.69       199
           1       0.70      0.67      0.69       201

    accuracy                           0.69       400
   macro avg       0.69      0.69      0.69       400
weighted avg       0.69      0.69      0.69       400



In [7]:

import tensorflow as tf
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


def build_dense_model(input_dim: int) -> tf.keras.Model:
    model = Sequential([
        Input(shape=(input_dim,)),             # Cleanly define input shape
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')         # Binary classification
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [8]:
# Convert lists to NumPy arrays
X_avg_train_np = np.array(X_avg_train)
X_avg_test_np = np.array(X_avg_test)
X_max_train_np = np.array(X_max_train)
X_max_test_np = np.array(X_max_test)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

print("Train labels:", np.unique(y_train_np, return_counts=True))
print("Test labels:", np.unique(y_test_np, return_counts=True))



Train labels: (array([0, 1]), array([801, 799]))
Test labels: (array([0, 1]), array([199, 201]))


In [9]:


if scale_data:
  from sklearn.preprocessing import StandardScaler

  scaler = StandardScaler()
  X_avg_train_np_scaled = scaler.fit_transform(X_avg_train_np)
  X_avg_test_np_scaled = scaler.transform(X_avg_test_np)
  print("Data Scaled!")
else:
  X_avg_train_np_scaled = X_avg_train_np
  X_avg_test_np_scaled = X_avg_test_np


# Train on average vectors
print("Training model on average vectors...")
model_avg = build_dense_model(embedding_dim)
# model_avg.fit(X_avg_train_np, y_train_np, epochs=10, batch_size=32, validation_split=0.1)
model_avg.fit(
    X_avg_train_np_scaled,
    y_train_np,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)


# Evaluate
print("Evaluating model on average vectors...")
loss, accuracy_avg = model_avg.evaluate(X_avg_test_np_scaled, y_test_np)
print(f"Test Accuracy (Average Vector): {accuracy_avg:.4f}")


Data Scaled!
Training model on average vectors...
Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5555 - loss: 0.7873 - val_accuracy: 0.7688 - val_loss: 0.5755
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6946 - loss: 0.5727 - val_accuracy: 0.8250 - val_loss: 0.4466
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7868 - loss: 0.4592 - val_accuracy: 0.8562 - val_loss: 0.3221
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8447 - loss: 0.3370 - val_accuracy: 0.8875 - val_loss: 0.2571
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8737 - loss: 0.2874 - val_accuracy: 0.8938 - val_loss: 0.2436
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9011 - loss: 0.2288 - val_accuracy: 0.9125 - val_loss: 0.2498
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9175 - loss: 0.1927 - val_accuracy: 0.9062 - val_loss: 0.2222
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9277

In [10]:


if scale_data:
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  X_max_train_np_scaled = scaler.fit_transform(X_max_train_np)
  X_max_test_np_scaled = scaler.transform(X_max_test_np)
else:
  X_max_train_np_scaled = X_max_train_np
  X_max_test_np_scaled = X_max_test_np



# Train on max-pooled vectors
print("Training model on max-pooled vectors...")
model_max = build_dense_model(embedding_dim)
model_max.fit(
    X_max_train_np_scaled,
    y_train_np,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
print("Evaluating model on max-pooled vectors...")
loss, accuracy_max = model_max.evaluate(X_max_test_np_scaled, y_test_np)
print(f"Test Accuracy (Max Pooling Vector): {accuracy_max:.4f}")


Training model on max-pooled vectors...
Epoch 1/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4841 - loss: 0.7676 - val_accuracy: 0.5875 - val_loss: 0.6774
Epoch 2/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5589 - loss: 0.6832 - val_accuracy: 0.6187 - val_loss: 0.6645
Epoch 3/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5839 - loss: 0.6666 - val_accuracy: 0.6625 - val_loss: 0.6383
Epoch 4/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6489 - loss: 0.6395 - val_accuracy: 0.7063 - val_loss: 0.6090
Epoch 5/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6939 - loss: 0.5903 - val_accuracy: 0.6750 - val_loss: 0.5865
Epoch 6/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7297 - loss: 0.5411 - val_accuracy: 0.6938 - val_loss: 0.5641
Epoch 7/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7668 - loss: 0.5071 - val_accuracy: 0.7125 - val_loss: 0.5498
Epoch 8/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7977 - loss: 0

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout

def build_sequence_model(seq_len: int, embedding_dim: int) -> tf.keras.Model:
    inputs = Input(shape=(seq_len, embedding_dim))
    x = Conv1D(64, kernel_size=3, activation='relu')(inputs)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [12]:
def review_to_sequences(words: List[str], seq_len: int = 10) -> List[np.ndarray]:
    """Convert a list of words into non-overlapping sequences of word vectors."""
    vectors = [w2v_model.wv[w.lower()] for w in words if w.lower() in w2v_model.wv]
    sequences = []
    for i in range(0, len(vectors) - seq_len + 1, seq_len):
        seq = vectors[i:i + seq_len]
        sequences.append(np.stack(seq))  # shape: (seq_len, embedding_dim)
    return sequences


In [13]:
sequence_length = 15


# Shuffle and split into train/test by review (not sequences)
train_reviews, test_reviews = train_test_split(documents, test_size=0.2, random_state=42)
# Sequence training data from train_reviews
X_seq, y_seq = [], []

for words, label in train_reviews:
    sequences = review_to_sequences(words, seq_len=sequence_length)
    X_seq.extend(sequences)
    y_seq.extend([1 if label == 'pos' else 0] * len(sequences))

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)


In [14]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

model_seq = build_sequence_model(sequence_length, embedding_dim)
model_seq.fit(
    X_seq_train, y_seq_train,
    epochs=30,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

loss, acc = model_seq.evaluate(X_seq_test, y_seq_test)
print(f"Sequence Model Accuracy: {acc:.4f}")


Epoch 1/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.5481 - loss: 0.6838 - val_accuracy: 0.6466 - val_loss: 0.6190
Epoch 2/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6596 - loss: 0.6187 - val_accuracy: 0.6666 - val_loss: 0.6068
Epoch 3/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6785 - loss: 0.5987 - val_accuracy: 0.6701 - val_loss: 0.6027
Epoch 4/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.6931 - loss: 0.5819 - val_accuracy: 0.6812 - val_loss: 0.5955
Epoch 5/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.7070 - loss: 0.5677 - val_accuracy: 0.6767 - val_loss: 0.5983
Epoch 6/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7182 - loss: 0.5530 - val_accuracy: 0.6647 - val_loss: 0.6031
Epoch 7/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7333 - loss: 0.5381 - val_accuracy: 0.6684 - val_loss: 0.6000
Epoch 8/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7351 - loss: 0.5296 - val_accura

In [15]:
def predict_review(words: List[str], model, seq_len: int = 10) -> float:
    """Predict review sentiment by averaging predictions over all sequences."""
    sequences = review_to_sequences(words, seq_len)
    if not sequences:
        return 0.5  # neutral prediction
    sequences_np = np.array(sequences)
    preds = model.predict(sequences_np, verbose=0)
    return float(np.mean(preds))  # average prediction across sequences


In [16]:
sample_review_words = movie_reviews.words(movie_reviews.fileids('pos')[0])
score = predict_review(words, model_seq, seq_len=sequence_length)
print(f"Predicted sentiment score: {score:.3f} → {'Positive' if score >= 0.5 else 'Negative'}")


Predicted sentiment score: 0.658 → Positive


In [17]:
# Run review-level evaluation
correct = 0
for words, label in test_reviews:
    score = predict_review(words, model_seq, seq_len=sequence_length)
    pred_label = 'pos' if score >= 0.5 else 'neg'
    if pred_label == label:
        correct += 1

review_accuracy = correct / len(test_reviews)
print(f"Review-level accuracy (Case C): {review_accuracy:.4f}")


Review-level accuracy (Case C): 0.8825
